# Projet Base de données

In [ ]:
import psycopg2
import psycopg2.extras
import pandas as pd
import threading
import time
import warnings
from db import connect
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

conn = connect()
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [ ]:
import init_db.drop as drop
import init_db.my_create_table as my_create_table

# creation de tables
drop.drop_tables(cur, conn)
my_create_table.create_tables(cur, conn)

In [ ]:
import init_db.insert_commune as insert_commune
import init_db.insert_stats_population as insert_stats_population
import init_db.insert_stats_mariage as insert_stats_mariage

# 2min - 2min30
insert_commune.fill_tables_commune()
print("On commence par insérer les données de population pour les années 1968 à 1999:")
insert_stats_population.fill_tables_population(start_year= 1968, end_year=1999) 
print("On insère les données de population pour les années 2000 à 2020:")
insert_stats_population.fill_tables_population(start_year= 2000, end_year=2020) 
insert_stats_mariage.fill_tables_mariage() 

## Ci-dessous 8 requêtes différentes:

1. Liste des départements d'une Region
2. Liste des communes de plus de X habitants d'un département donné
3. La région la plus peuplé
4. Le département le plus peuplé d'une région donnée
5. Le nombre de mariages par région en 2021
6. Le nombre de mariages par tranche d'âge dans une région donnée en 2021
7. Le nombre de mariages où au moins un des conjoints est un étranger, par région
8. Densité de la population (nombre d'habitants au km²) en 2020 par région



### Liste des départements d'une Region

In [ ]:
nom_region = "Nouvelle-Aquitaine"

request = f"""
SELECT d.id_departement, d.nom_departement
FROM departement d
JOIN region r ON d.id_region = r.id_region
WHERE r.nom_region = '{nom_region}';
"""

cur.execute(request)
rows = cur.fetchall()

df = pd.DataFrame(rows, columns=['id_departement', 'nom_departement'])
df

### Liste des communes de plus de X habitants d'un département donné

In [ ]:
# Code du département et nombre minimum d'habitants
id_departement = "24"
min_population = 10_000

request = f"""
SELECT c.id_commune, c.nom_commune, sp.valeur AS population
FROM commune c
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE c.id_departement = '{id_departement}'
  AND sp.annee = 2020
  AND sp.type_statistique = 'Population'
  AND sp.valeur > {min_population};
"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_commune', 'nom_commune', 'population'])
df

### La région la plus peuplé

In [ ]:
request = """
SELECT r.id_region, r.nom_region, SUM(sp.valeur) AS population_totale
FROM region r
JOIN departement d ON r.id_region = d.id_region
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.annee = 2020 AND sp.type_statistique = 'Population'
GROUP BY r.id_region, r.nom_region
ORDER BY population_totale DESC
LIMIT 1;
"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'population_totale'])
df


### Le département le plus peuplé d'une région donnée

In [ ]:
nom_region = 'Occitanie'

request = f"""SELECT d.id_departement, d.nom_departement, SUM(sp.valeur) AS population_totale
FROM departement d
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE d.id_region = (SELECT id_region FROM region WHERE nom_region = '{nom_region}')
  AND sp.annee = 2020 AND sp.type_statistique = 'Population'
GROUP BY d.id_departement, d.nom_departement
ORDER BY population_totale DESC
LIMIT 1;
"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_departement', 'nom_departement', 'population_totale'])
df

### Le nombre de mariages par région

In [ ]:
request = f"""SELECT r.id_region, r.nom_region, SUM(sm.nb_mariages) AS total_mariages
FROM region r
JOIN statistiques_mariages_age sm ON r.id_region = sm.id_region
WHERE sm.annee = 2021
GROUP BY r.id_region, r.nom_region
ORDER BY total_mariages DESC;"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'total_mariages'])
df

### Le nombre de mariages par tranche d'âge dans une région donnée

In [ ]:
nom_region = 'Nouvelle-Aquitaine'

request = f"""SELECT sma.grage AS tranche_age, SUM(sma.nb_mariages) AS total_mariages
FROM statistiques_mariages_age sma
JOIN region r ON sma.id_region = r.id_region
WHERE sma.annee = 2021 AND r.nom_region = '{nom_region}'
GROUP BY sma.grage
ORDER BY sma.grage;
"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['tranche_age', 'total_mariages'])
df

### Le nombre de mariages où au moins un des conjoints est un étranger, par région

In [ ]:
request = """SELECT r.id_region, r.nom_region, SUM(sme.nb_mariages_nationalite) AS total_mariages
FROM region r
JOIN statistiques_mariages_origine sme ON r.id_region = sme.id_region
WHERE sme.annee = 2021 AND sme.code IN ('FR_ETR', 'ETR_ETR')
GROUP BY r.id_region, r.nom_region
ORDER BY total_mariages DESC;"""
cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'total_mariages'])
df

### Densité de la population (nombre d'habitants au km²) en 2020 par région

In [ ]:
request = """
SELECT 
    r.id_region, 
    r.nom_region, 
    COALESCE(SUM(sp.valeur), 0) AS population_totale,
    COALESCE(SUM(c.superf), 0) AS superficie_totale,
    CASE 
        WHEN SUM(c.superf) > 0 THEN ROUND((SUM(sp.valeur) / SUM(c.superf))::numeric, 2)
        ELSE 0.00 
    END AS habitants_par_km2
FROM 
    region r
JOIN 
    departement d ON r.id_region = d.id_region
JOIN 
    commune c ON d.id_departement = c.id_departement
JOIN 
    statistiques_population sp ON c.id_commune = sp.codgeo AND sp.type_statistique = 'Population' AND sp.annee = 2020
GROUP BY 
    r.id_region, r.nom_region
ORDER BY 
    habitants_par_km2 DESC;
"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'population_totale', 'superficie_totale', 'habitants_par_km2'])
df

## Views

Créer deux vues (cf commande CREATE OR REPLACE VIEW) qui donnent la population des départements et des régions pour les différentes années ainsi que les indicateurs existants.

### Vue 1 : Population des départements

In [ ]:
vue = """
CREATE OR REPLACE VIEW vue_population_departement AS
SELECT d.id_departement, d.nom_departement, sp.annee, sp.type_statistique, SUM(sp.valeur) AS population_totale
FROM departement d
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.type_statistique = 'Population'
GROUP BY d.id_departement, d.nom_departement, sp.annee, sp.type_statistique;
"""

cur.execute(vue)

request = """
SELECT *
FROM vue_population_departement
WHERE annee = 2020
ORDER BY population_totale DESC;
"""
cur.execute(request)
conn.commit()
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_departement', 'nom_departement', 'annee', 'type_statistique', 'population_totale'])
df


### Vue 2 : Population des régions

In [ ]:
vue_region = """
CREATE OR REPLACE VIEW vue_population_region AS
SELECT r.id_region, r.nom_region, sp.annee, sp.type_statistique, SUM(sp.valeur) AS population_totale
FROM region r
JOIN departement d ON r.id_region = d.id_region
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.type_statistique = 'Population'
GROUP BY r.id_region, r.nom_region, sp.annee, sp.type_statistique;
"""

try:
    cur.execute(vue_region)
    conn.commit()  
    
    request = """
    SELECT *
    FROM vue_population_region
    ORDER BY population_totale DESC;
    """
    
    cur.execute(request)
    rows = cur.fetchall()
    df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'annee', 'type_statistique', 'population_totale'])
    
except Exception as e:
    conn.rollback() 
    print(f"Error: {e}")
    
df

### Vue Bonus: Details sur les régions

In [ ]:
try:
    cur.execute("DROP VIEW IF EXISTS vue_region_detaillee;")
    conn.commit()

    vue_region_details = """
    CREATE OR REPLACE VIEW vue_region_detaillee AS
    SELECT 
        r.id_region,
        r.nom_region,
        COALESCE(pop.population_totale, 0) AS population_totale,
        COALESCE(pop.superficie_totale, 0) AS superficie_totale,
        CASE 
            WHEN pop.superficie_totale > 0 THEN ROUND((pop.population_totale / pop.superficie_totale)::numeric, 2)
            ELSE 0
        END AS densite_population,
        COALESCE(mar.mariages_hf_h, 0) AS mariages_hf_h,
        COALESCE(mar.mariages_hf_f, 0) AS mariages_hf_f,
        COALESCE(mar.mariages_hh, 0) AS mariages_hh,
        COALESCE(mar.mariages_ff, 0) AS mariages_ff
    FROM 
        region r
    LEFT JOIN (
        SELECT
            d.id_region,
            SUM(sp.valeur) AS population_totale,
            SUM(c.superf) AS superficie_totale
        FROM 
            departement d
        JOIN 
            commune c ON d.id_departement = c.id_departement
        LEFT JOIN 
            statistiques_population sp ON c.id_commune = sp.codgeo AND sp.type_statistique = 'Population' AND sp.annee = 2020
        GROUP BY 
            d.id_region
    ) pop ON r.id_region = pop.id_region
    LEFT JOIN (
        SELECT
            sm.id_region,
            SUM(CASE WHEN sm.typmar3 = 'HF-H' THEN sm.nb_mariages ELSE 0 END) AS mariages_hf_h,
            SUM(CASE WHEN sm.typmar3 = 'HF-F' THEN sm.nb_mariages ELSE 0 END) AS mariages_hf_f,
            SUM(CASE WHEN sm.typmar3 = 'HH' THEN sm.nb_mariages ELSE 0 END) AS mariages_hh,
            SUM(CASE WHEN sm.typmar3 = 'FF' THEN sm.nb_mariages ELSE 0 END) AS mariages_ff
        FROM 
            statistiques_mariages_age sm
        WHERE
            sm.annee = 2021
        GROUP BY 
            sm.id_region
    ) mar ON r.id_region = mar.id_region;
    """
    cur.execute(vue_region_details)
    conn.commit()

    request = """
    SELECT 
        nom_region,
        population_totale,
        densite_population,
        mariages_hf_h,
        mariages_hf_f,
        mariages_hh,
        mariages_ff
    FROM 
        vue_region_detaillee
    WHERE 
        densite_population > 100
    ORDER BY 
        densite_population DESC;
    """
    cur.execute(request)
    rows = cur.fetchall()
    df = pd.DataFrame(rows, columns=['nom_region', 'population_totale', 'densite_population', 'mariages_hf_h', 'mariages_hf_f', 'mariages_hh', 'mariages_ff'])
    
except Exception as e:
    conn.rollback() 
    print(f"Error: {e}")
finally:
    conn.commit()

df

In [ ]:
try:
    cur.execute("DROP VIEW IF EXISTS vue_mariages_par_region;")
    conn.commit()
    
    
    view = """
    CREATE OR REPLACE VIEW vue_mariages_par_region AS
    SELECT 
        r.id_region,
        r.nom_region,
        sma.annee,
        sma.typmar3,
        sma.grage AS tranche_age,
        SUM(sma.nb_mariages) AS total_mariages
    FROM 
        region r
    JOIN 
        statistiques_mariages_age sma ON r.id_region = sma.id_region
    GROUP BY 
        r.id_region, r.nom_region, sma.annee, sma.typmar3, sma.grage
    ORDER BY 
        r.nom_region, sma.annee, sma.typmar3, sma.grage;
    """
    
    cur.execute(view)
    conn.commit()
    
    request = """
    SELECT *
    FROM vue_mariages_par_region
    WHERE annee = 2021
    ORDER BY nom_region, typmar3, tranche_age;
    """
    
    cur.execute(request)
    rows = cur.fetchall()
    df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'annee', 'typmar3', 'tranche_age', 'total_mariages'])
    
except Exception as e:
    conn.rollback() 
    print(f"Error: {e}")
finally:
    conn.commit()
    
df

Vue sur les logements TODO

## Procédure stockée

In [ ]:
alter_departements = "ALTER TABLE departement ADD COLUMN population_totale INT;"
alter_regions = "ALTER TABLE region ADD COLUMN population_totale INT;"

cur.execute(alter_departements)
cur.execute(alter_regions)
conn.commit()

# Créer la procédure stockée
procedure_calcul = """
CREATE OR REPLACE PROCEDURE calculer_population()
LANGUAGE plpgsql
AS $$
BEGIN
    -- Calculer la population des départements
    UPDATE departement d
    SET population_totale = sub.population
    FROM (
        SELECT c.id_departement, SUM(sp.valeur) AS population
        FROM commune c
        JOIN statistiques_population sp ON c.id_commune = sp.codgeo
        WHERE sp.type_statistique = 'Population' AND sp.annee = 2020
        GROUP BY c.id_departement
    ) AS sub
    WHERE d.id_departement = sub.id_departement;

    -- Calculer la population des régions
    UPDATE region r
    SET population_totale = sub.population
    FROM (
        SELECT d.id_region, SUM(d.population_totale) AS population
        FROM departement d
        GROUP BY d.id_region
    ) AS sub
    WHERE r.id_region = sub.id_region;
END;
$$;
"""

cur.execute(procedure_calcul)
conn.commit()

cur.execute("CALL calculer_population();")
conn.commit()


In [ ]:

request = """
SELECT * FROM departement;
"""
cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_departement', 'nom_departement', 'id_region', 'population_totale'])
df

## Triggers

In [ ]:
try:
    # Création du trigger pour empêcher les modifications dans la table region
    bloque_region = """
    CREATE OR REPLACE FUNCTION bloque_actions_region()
    RETURNS trigger AS $$
    BEGIN
        IF TG_OP = 'DELETE' OR TG_OP = 'INSERT' THEN
            RAISE EXCEPTION 'Insertion ou suppression non autorisée dans la table region';
        ELSIF TG_OP = 'UPDATE' THEN
            IF OLD.id_region IS DISTINCT FROM NEW.id_region OR OLD.nom_region IS DISTINCT FROM NEW.nom_region THEN
                RAISE EXCEPTION 'Modification de la table region non autorisée sur les colonnes id_region ou nom_region';
            END IF;
        END IF;
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;

    CREATE TRIGGER tr_bloquer_actions_region
    BEFORE INSERT OR UPDATE OR DELETE ON region
    FOR EACH ROW EXECUTE FUNCTION bloque_actions_region();
    """
    
    # Création du trigger pour empêcher les modifications dans la table departement
    bloque_departement = """
    CREATE OR REPLACE FUNCTION bloque_actions_departement()
    RETURNS trigger AS $$
    BEGIN
        IF TG_OP = 'DELETE' OR TG_OP = 'INSERT' THEN
            RAISE EXCEPTION 'Insertion ou suppression non autorisée dans la table departement';
        ELSIF TG_OP = 'UPDATE' THEN
            IF OLD.id_departement IS DISTINCT FROM NEW.id_departement OR 
            OLD.nom_departement IS DISTINCT FROM NEW.nom_departement OR
            OLD.id_region IS DISTINCT FROM NEW.id_region THEN
                RAISE EXCEPTION 'Modification de la table departement non autorisée sur les colonnes id_departement, nom_departement ou id_region';
            END IF;
        END IF;
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;

    CREATE TRIGGER tr_bloquer_actions_departement
    BEFORE INSERT OR UPDATE OR DELETE ON departement
    FOR EACH ROW EXECUTE FUNCTION bloque_actions_departement();
    """
    
    # Création de la procédure pour mettre à jour les populations
    procedure_calcul = """
    CREATE OR REPLACE PROCEDURE calculer_population()
    LANGUAGE plpgsql
    AS $$
    BEGIN
        UPDATE departement d
        SET population_totale = sub.population
        FROM (
            SELECT c.id_departement, SUM(sp.valeur) AS population
            FROM commune c
            JOIN statistiques_population sp ON c.id_commune = sp.codgeo
            WHERE sp.type_statistique = 'Population' AND sp.annee = 2020
            GROUP BY c.id_departement
        ) AS sub
        WHERE d.id_departement = sub.id_departement;

        -- Calculer la population des régions
        UPDATE region r
        SET population_totale = sub.population
        FROM (
            SELECT d.id_region, SUM(d.population_totale) AS population
            FROM departement d
            GROUP BY d.id_region
        ) AS sub
        WHERE r.id_region = sub.id_region;
    END;
    $$;
    """
    
    # Création du trigger pour mettre à jour les populations
    maj_population = """
    CREATE OR REPLACE FUNCTION maj_population()
    RETURNS trigger AS $$
    BEGIN
        CALL calculer_population();
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;

    CREATE TRIGGER tr_maj_population
    AFTER INSERT OR UPDATE ON statistiques_population
    FOR EACH ROW EXECUTE FUNCTION maj_population();
    """

    cur.execute(bloque_region)
    cur.execute(bloque_departement)
    cur.execute(procedure_calcul)
    cur.execute(maj_population)
    
    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Error: {e}")


Voici quelques requêtes qui déclenchent les triggers:

In [ ]:
try:
    cur.execute("INSERT INTO region (id_region, nom_region) VALUES (100, 'Nouvelle Région')")
    conn.commit()
except Exception as e:
    conn.rollback()
    print("Erreur lors de la tentative de modification de la table 'region':", e)


In [ ]:
try:
    cur.execute("UPDATE departement SET nom_departement = 'Département Modifié' WHERE id_departement = '33'")
    conn.commit()
except Exception as e:
    conn.rollback()
    print("Erreur lors de la tentative de modification de la table 'departement':", e)

## Triggers suite

In [ ]:
try:
    # Création de la procédure pour mettre à jour la population d'un département
    procedure_update_departement = """
    CREATE OR REPLACE PROCEDURE update_population_departement()
    LANGUAGE plpgsql
    AS $$
    BEGIN
        UPDATE departement d
        SET population_totale = sub.population
        FROM (
            SELECT c.id_departement, SUM(sp.valeur) AS population
            FROM commune c
            JOIN statistiques_population sp ON c.id_commune = sp.codgeo
            WHERE sp.type_statistique = 'Population' AND sp.annee IN (2020, 2021, 2022, 2023)
            GROUP BY c.id_departement
            HAVING COUNT(c.id_commune) = (
                SELECT COUNT(*)
                FROM commune
                WHERE id_departement = c.id_departement
            )
        ) AS sub
        WHERE d.id_departement = sub.id_departement;
    END;
    $$;
    """

    # Création de la procédure pour mettre à jour la population d'une région
    procedure_update_region = """
    CREATE OR REPLACE PROCEDURE update_population_region()
    LANGUAGE plpgsql
    AS $$
    BEGIN
        UPDATE region r
        SET population_totale = sub.population
        FROM (
            SELECT d.id_region, SUM(d.population_totale) AS population
            FROM departement d
            GROUP BY d.id_region
            HAVING COUNT(d.id_departement) = (
                SELECT COUNT(*)
                FROM departement
                WHERE id_region = d.id_region
            )
        ) AS sub
        WHERE r.id_region = sub.id_region;
    END;
    $$;
    """

    # Création de la procédure qui met à jour les populations des départements et des régions
    procedure_update_all = """
    CREATE OR REPLACE PROCEDURE update_population_all()
    LANGUAGE plpgsql
    AS $$
    BEGIN
        CALL update_population_departement();
        CALL update_population_region();
    END;
    $$;
    """

    # Création du trigger pour mettre à jour les populations
    trigger_update_population = """
    CREATE OR REPLACE FUNCTION trigger_population_update()
    RETURNS trigger AS $$
    BEGIN
        CALL update_population_all();
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;

    CREATE TRIGGER tr_update_population
    AFTER INSERT OR UPDATE ON statistiques_population
    FOR EACH ROW EXECUTE FUNCTION trigger_population_update();
    """

    cur.execute(procedure_update_departement)
    cur.execute(procedure_update_region)
    cur.execute(procedure_update_all)
    cur.execute(trigger_update_population)

    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Error: {e}")
    

# Declencher le trigger de mise à jour des populations
# Quand on insert une nouvelle donnée 
# où type_statistique = 'Population'
# Aussi essayer de voir pour faire la maj quand toutes les sttaistiques sont insérées et pas avant
# Donc attendre d'avoir toutes les communes d'un département avant de faire la maj de la population du département
# et attendre d'avoir tous les départements d'une région avant de faire la maj de la population de la région

In [ ]:
try:
    # Sauvegarder l'ancienne valeur pour restauration ultérieure
    cur.execute("SELECT valeur FROM statistiques_population WHERE codgeo = '33063' AND annee = 2020 AND type_statistique = 'Population'")
    old_value = cur.fetchone()[0]

    # Mettre à jour la population de Bordeaux à 1 milliard
    cur.execute("UPDATE statistiques_population SET valeur = 1000000000 WHERE codgeo = '33063' AND annee = 2020 AND type_statistique = 'Population'")
    conn.commit()

    # Vérifier les nouvelles valeurs pour la Gironde et la Nouvelle-Aquitaine
    cur.execute("SELECT nom_departement, population_totale FROM departement WHERE id_departement = '33'")
    gironde_population = cur.fetchone()
    cur.execute("SELECT nom_region, population_totale FROM region WHERE id_region = '75'")
    nouvelle_aquitaine_population = cur.fetchone()

    print(f"Nouvelle population de la Gironde: {gironde_population}")
    print(f"Nouvelle population de la Nouvelle-Aquitaine: {nouvelle_aquitaine_population}")

    # Restaurer la valeur originale pour Bordeaux
    cur.execute("UPDATE statistiques_population SET valeur = %s WHERE codgeo = '33063' AND annee = 2020 AND type_statistique = 'Population'", (old_value,))
    conn.commit()

except Exception as e:
    print(f"Erreur: {e}")
    conn.rollback()

In [ ]:
# On véririfie que l'ancienne population de Bordeaux a bien été restaurée
cur.execute("SELECT valeur FROM statistiques_population WHERE codgeo = '33063' AND annee = 2020 AND type_statistique = 'Population'")
old_value = cur.fetchone()[0]
print(f"Population de Bordeaux: {old_value}")


## Plan d'exécution (EXPLAIN)
### La région la plus peuplé

In [ ]:
request = """
EXPLAIN ANALYSE SELECT r.id_region, r.nom_region, SUM(sp.valeur) AS population_totale
FROM region r
JOIN departement d ON r.id_region = d.id_region
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.annee = 2020 AND sp.type_statistique = 'Population'
GROUP BY r.id_region, r.nom_region
ORDER BY population_totale DESC
LIMIT 1;
"""

cur.execute(request)
rows = cur.fetchall()
for row in rows:
    print(row[0])

Dans l'output ci-dessus, vous pouvez voir que le coût total estimé pour la requête est de 15108.32 unités. Cependant, le coût réel d'exécution peut être différent, car les estimations de coût sont basées sur les statistiques de la base de données, qui peuvent ne pas être parfaitement précises.

Dans notre cas, le coût réel d'exécution de la requête était d'environ 104,881 ms (Execution Time: 104.881 ms). Cela signifie que la requête a pris un peu plus de 100 millisecondes pour s'exécuter.

On peut voir que PostgreSQL a utilisé différents algorithmes de jointure et de tri pour exécuter la requête.

**Algorithmes de jointure**

1. **Hash Join** : Cet algorithme a été utilisé pour joindre les tables `statistiques_population` et `commune` sur la colonne `id_commune`. Il a également été utilisé pour joindre les tables `commune` et `departement` sur la colonne `id_departement`.
2. **Nested Loop Join** : Cet algorithme a été utilisé pour joindre les tables `departement` et `region` sur la colonne `id_region`.

**Stratégies de tri**

1. **Sort Method: quicksort** : Cette stratégie de tri a été utilisée pour trier les résultats intermédiaires de la jointure entre les tables `statistiques_population`, `commune`, `departement` et `region`.
2. **Top-N heapsort** : Cette stratégie de tri a été utilisée pour trier les résultats finaux de la requête, en utilisant l'ordre de tri spécifié dans la clause `ORDER BY` et en limitant le nombre de résultats à 1.

Le choix de l'algorithme de jointure ou de tri dépend de plusieurs facteurs, tels que la taille des tables, la présence ou non d'index sur les colonnes de jointure, et les contraintes de performance spécifiées dans la requête. Dans ce cas, PostgreSQL a choisi les algorithmes et les stratégies de tri les plus efficaces pour exécuter la requête en fonction des données et des contraintes spécifiées. 

Afin de montrer l'intérêt des index, nous allons en implementer quelques-uns avant de comparer leurs performances.

## Plan d'exécution et index

### La région la plus peuplé avec index

In [ ]:
request = """
CREATE INDEX pop_idx ON statistiques_population (valeur) WHERE type_statistique = 'Population';
EXPLAIN ANALYSE SELECT r.id_region, r.nom_region, SUM(sp.valeur) AS population_totale
FROM region r
JOIN departement d ON r.id_region = d.id_region
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.annee = 2020 AND sp.type_statistique = 'Population'
GROUP BY r.id_region, r.nom_region
ORDER BY population_totale DESC
LIMIT 1;
"""

cur.execute(request)
rows = cur.fetchall()
for row in rows:
    print(row[0])

On peut voir que l'utilisation d'un index a permis d'améliorer les performances de la requête.

Sans l'index, la requête a dû parcourir l'intégralité de la table `statistiques_population` pour trouver les lignes correspondant à la condition `type_statistique = 'Population'`. Cela aurait nécessité un séquentiel scan complet de la table, ce qui peut être coûteux en termes de temps et de ressources, en particulier si la table est volumineuse.

Avec l'index, la requête a pu utiliser un bitmap index scan pour trouver rapidement les lignes correspondant à la condition `type_statistique = 'Population'`. Cela a permis de réduire considérablement le nombre de lignes à examiner, ce qui a accéléré la requête.

On peut voir dans l'output que le bitmap index scan a pris seulement 11,554 ms pour renvoyer les 104 583 lignes correspondant à la condition, alors qu'un séquentiel scan complet de la table aurait pris beaucoup plus de temps.

Ensuite, la requête a utilisé un hash join pour joindre les tables `statistiques_population`, `commune`, `departement` et `region`, en utilisant les clés primaires et étrangères pour relier les lignes correspondantes. L'utilisation d'un index sur la colonne `id_commune` de la table `statistiques_population` a permis d'accélérer la jointure en utilisant un hash join plus efficace.

Enfin, la requête a utilisé un tri top-N heapsort pour trier les résultats finaux en fonction de la population totale, et a renvoyé le premier résultat en utilisant une limite de 1.

En comparant cet output à celui sans index, on peut voir que l'utilisation d'index a permis d'améliorer considérablement les performances de la requête, en réduisant le temps d'exécution de plus de la moitié (de 290.121 ms à 109.404 ms remarque: ces valeurs susceptibles de varier). Cela démontre l'importance des index dans l'optimisation des requêtes SQL.

## Pour aller plus loin : transactions

Prendre deux clients (deux clients en console ou un client web et un client console) pour voir que les modifs faites sur un ne sont pas visibles immédiatement. Tester et illustrer avec des exemples les différents niveaux d'isolation.

In [ ]:
def add_population_entry(delay):
    conn = connect()
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_READ_COMMITTED)
    cur = conn.cursor()
    try:
        cur.execute("BEGIN;")
        cur.execute("INSERT INTO statistiques_population (codgeo, annee, type_statistique, valeur) VALUES ('33063', 2024, 'TEST', 150000);")
        print("Transaction A: Insertion réalisée, en attente de commit...")
        time.sleep(delay) 
        conn.commit()
        print("Transaction A: Population ajoutée et commitée.")
    except Exception as e:
        print(f"Transaction A: Erreur lors de l'ajout de population, {e}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()

def read_population_entry(delay):
    time.sleep(delay)  # Attend que la transaction A commence
    conn = connect()
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_READ_COMMITTED)
    cur = conn.cursor()
    try:
        cur.execute("SELECT valeur FROM statistiques_population WHERE codgeo = '33063' AND annee = 2024 AND type_statistique = 'TEST';")
        result = cur.fetchone()
        if result:
            print(f"Transaction B: Population vue par B: {result[0]}")
        else:
            print("Transaction B: Aucune population trouvée.")
    except Exception as e:
        print(f"Transaction B: Erreur lors de la lecture de population, {e}")
    finally:
        cur.close()
        conn.close()

# Lancer les transactions en utilisant des threads
thread_a = threading.Thread(target=add_population_entry, args=(10,))
thread_b = threading.Thread(target=read_population_entry, args=(5,))

thread_a.start()
thread_b.start()

thread_a.join()
thread_b.join()
